View Graph 

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import altair as alt
import locale

# set locale time to Spanish for date parsing
locale.setlocale(locale.LC_TIME, "es_ES.UTF-8")  # for Linux/Mac
# locale.setlocale(locale.LC_TIME, "Spanish_Spain.1252")  # for Windows

# URL of the webpage containing the table
url = "https://www.hacienda.cl/areas-de-trabajo/finanzas-internacionales/oficina-de-la-deuda-publica/plan-emision"

response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
#locate table in the page
table = soup.find("table", {"class": "tableizer-table"})

headers = [header.get_text(strip=True) for header in table.find_all("th")]
rows = []
for row in table.find_all("tr")[1:]:  # Skip the header row
    cells = row.find_all("td")
    rows.append([cell.get_text(strip=True) for cell in cells])

# data frame from the HTML 
df = pd.DataFrame(rows, columns=headers)

# cleaning. Coercing date format
df["Fecha Colocación"] = pd.to_datetime(df["Fecha Colocación"], format="%d-%b-%y", errors='coerce', dayfirst=True)
df["Vencimiento"] = pd.to_datetime(df["Vencimiento"], format="%d-%b-%y", errors='coerce', dayfirst=True)
df["Colocado (Moneda Original)"] = df["Colocado (Moneda Original)"].str.replace(".", "", regex=False).astype(float)

# Transforming chilean pesos to USD
usd_to_clp_rate = 1007.04
#jan 6 2024 change

df["Placed (USD)"] = df["Colocado (Moneda Original)"]/usd_to_clp_rate

filtered_df = df[df["Bono"].str.contains("BTP", na=False)]
filtered_df["Vencimiento"] = pd.to_datetime(filtered_df["Vencimiento"], errors="coerce")

filtered_df["Year of Maturity"] = filtered_df["Vencimiento"].dt.year # creatting year of maturity variable

# grouped frame to plot
grouped_df = filtered_df.groupby("Year of Maturity")["Placed (USD)"].sum().reset_index() 

#tets vega plot with altair 
line_chart = alt.Chart(grouped_df).mark_line().encode(
    x=alt.X("Year of Maturity:O", title="Year of Maturity"),
    y=alt.Y("Placed (USD):Q", title="Total Amount of Bonds Maturing (USD)"),
    tooltip=["Year of Maturity", "Placed (USD)"]
).properties(
    title="Trend of Government Bond Maturities by Year (Only bonds issued in 2024)",
    width=800,
    height=400
)

# Display the line chart
line_chart.display()

/var/folders/7b/vjmsjgqx12j148x13wgz79cr0000gn/T/ipykernel_98130/2547992761.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["Vencimiento"] = pd.to_datetime(filtered_df["Vencimiento"], errors="coerce")
/var/folders/7b/vjmsjgqx12j148x13wgz79cr0000gn/T/ipykernel_98130/2547992761.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["Year of Maturity"] = filtered_df["Vencimiento"].dt.year # creatting year of maturity variable


alt.Chart(...)

export graph as .json vega lite grpah

In [3]:
import json

#create vega plot in json
data_json = grouped_df.to_dict(orient="records")

chart_5 = {
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": {
      "text": "Trend of Government Bond Maturities by Year, Chile",
      "anchor": "start",
      "subtitle": "Only bonds issued in 2024 | Source: Finance ministery of Chile",
      "fontSize": "17",
      "subtitleFontSize": "14",
      "color": "#000000",
      "subtitleColor": "#000000"
    },
    "height": "400",
    "width": "600",
    "autosize": {
      "type": "fit",
      "contains": "padding"
    },
    "background": "#ffffff",
    "view": {
      "fill": "#ffffff",
      "strokeOpacity": 0
    },

    "data": {
        "values": data_json 
    },
    "mark": {
    "type": "line",
    "color": "red",
    "strokeWidth": 3
  },
    "encoding": {
        "x": {"field": "Year of Maturity", "type": "ordinal", "title": "Year of Maturity", 
            "axis": {
            "title": "Year",
            "labelAngle": 0,
            "labelFontSize": 12,
            "titleFontSize": 14
      }},
        "y": {"field": "Placed (USD)", "type": "quantitative", "title": "Total Amount placed (USD)",
            "axis": {
                "title": "Total Amount of Bonds Maturing (USD)",
                "labelAngle": 0,
                "labelFontSize": 12,
                "titleFontSize": 14
            },
        "tooltip": [
            {"field": "Year of Maturity", "type": "ordinal"},
            {"field": "Placed (USD)", "type": "quantitative"}
        ]
    }
}
}

json_filename = "chart_5.json"
with open(json_filename, "w") as f:
    json.dump(chart_5, f, indent=4)